In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Q1 Sobel

## import part

In [3]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import sys

## a)

In [19]:
# generate random matrix with size n*n
def generateRandomMatrix(n):
  matrix = np.random.randint(0, 256, size=(n,n), dtype=np.uint8)
  return matrix

In [33]:
# write convolution function
def convolve(matrix, kernel):
  kernel_height, kernel_width = kernel.shape
  matrix_paded = cv2.copyMakeBorder(matrix, kernel_height//2, kernel_height//2, kernel_width//2, kernel_width//2, cv2.BORDER_REFLECT)
  temp_out = np.zeros_like(matrix)

  for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
      sel_region = matrix_paded[i:i+kernel_width, j:j+kernel_height]
      temp_out[i, j] = np.sum(sel_region * kernel)

  return temp_out

In [4]:
# generate horizontal and vertical kernels
horizontal_sobel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
vertical_sobel = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

In [20]:
# convolve the initial matrix with kernels
def util_convolve(matrix, sobel_operator):
  result = np.zeros_like(matrix)
  result = convolve(matrix, sobel_operator)
  return result

# create the two array for save the horizontal and vertical derivatives
init_matrix = generateRandomMatrix(10)
horizontal_out = util_convolve(init_matrix, horizontal_sobel)
vertical_out = util_convolve(init_matrix, vertical_sobel)

In [ ]:
# generate magnitude and direction matrixes
def calc_magnitude(h_out, v_out):
  return np.sqrt(h_out**2 + v_out**2)
def calc_direction(h_out, v_out):
  return np.arctan2(v_out, h_out)

print("Magnitude Matrix:\n", calc_magnitude(horizontal_out, vertical_out))
print("Direction Matrix:\n", calc_direction(horizontal_out, vertical_out))

## b)

In [6]:
# load image and convert it to gray scale
path = '/content/drive/MyDrive/ColabNotebooks/FCV/HW2/images/afshin.jpg'
img = cv2.imread(path)
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [7]:
# define gaussian kernel
def generate_gussian_kernel(kernel_size, sigma):
  kernel = np.zeros((kernel_size, kernel_size))
  for i in range(kernel_size):
    for j in range(kernel_size):
      s = i - kernel_size // 2
      t = j - kernel_size // 2
      kernel[i, j] = np.exp(-(s ** 2 + t ** 2) / (2 * sigma ** 2))
  
  # normalize the kernel
  return kernel/np.sum(kernel)

In [ ]:
# generate Sobel outputs for 2 cases with gaussion filter and without gaussion filter and compare them
# first show the Sobel output for case that we don't use any smoothing filters
x_out = convolve(gray_image, horizontal_sobel)
y_out = convolve(gray_image, vertical_sobel)
cv2_imshow(calc_magnitude(x_out, y_out))

# show Sobel output for case that we use gussian filter for smoothing
smooth_img = cv2.filter2D(gray_image, -1, generate_gussian_kernel(3, 40))
x2_out = cv2.filter2D(smooth_img, -1, horizontal_sobel)
y2_out = cv2.filter2D(smooth_img, -1, vertical_sobel)
cv2_imshow(calc_magnitude(x2_out, y2_out))

## c)

In [ ]:
# do the operations in part b with OpenCV Sobel method and describe its parameters
sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=1, scale=1, delta=0)
sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=1, scale=1, delta=0)

# Combine the results of the Sobel filter in both x and y directions
sobel = calc_magnitude(sobelx, sobely)
cv2_imshow(sobel)